# 📓 The GenAI Revolution Cookbook

**Title:** How to Build Multi-Agent AI Systems with CrewAI, Step by Step

**Description:** Design scalable multi-agent AI systems with CrewAI using YAML configs, task delegation, tools, and training—then build a Customer Feedback Analyst.

---

*This jupyter notebook contains executable code examples. Run the cells below to try out the code yourself!*



Generative AI gets useful when it ships. In this guide, you'll build a Customer Feedback Analyst using CrewAI. You'll define agents in YAML, wire tasks and tools, run the system end-to-end, and produce a real report with sentiment tables and a pie chart. You'll learn the exact architecture and code to reuse on your own data. For those interested in building reliable pipelines for extracting structured data with LLMs, our [structured data extraction pipeline guide](/article/structured-data-extraction-with-llms-how-to-build-a-pipeline-3) provides a detailed walkthrough of best practices.

## Why This Approach Works

Multi-agent systems shine when you need specialized roles working together. A single LLM call can classify sentiment, but it struggles to coordinate reading files, computing aggregates, generating charts, and assembling reports in one pass. By splitting these responsibilities across focused agents, you gain modularity, clarity, and the ability to swap models or tools per task.

CrewAI provides a clean model for agents, tasks, tools, and crews. You define roles and workflows in YAML, reduce boilerplate, and run sequential, hierarchical, or parallel processes with built-in context passing, delegation, and memory. To further standardize tool and data access across agents and environments, consider learning about the [Model Context Protocol (MCP)](/article/model-context-protocol-mcp-explained-2025-guide-for-builders), which can help your agents interoperate and audit more effectively.

This tutorial uses a sequential pipeline: each task receives context from previous steps, keeping logic simple and traceable. You'll wire a file reader, a sentiment classifier, an aggregation tool, a chart generator, and a report writer into a single crew that runs end-to-end.

## How It Works

The pipeline has five stages:

1. **Read feedback**: A file reader tool loads the CSV.
2. **Classify sentiment**: An agent labels each row as positive, neutral, or negative, outputting JSONL.
3. **Aggregate counts**: A Python tool parses the JSONL and computes sentiment totals deterministically.
4. **Generate chart**: A tool creates a pie chart PNG from the aggregated counts.
5. **Assemble report**: A writer agent combines the table and chart into a Markdown document.

Each agent has a role, goal, backstory, and assigned tools. Tasks declare dependencies via context, ensuring outputs flow in order. The crew orchestrates execution, and you get a final report with real artifacts.

## What You'll Build

By the end, you'll have:

- A `customer_feedback_report.md` with sentiment breakdown and insights.
- A `sentiment_pie.png` chart visualizing the distribution.
- A reusable pipeline you can adapt to other CSV datasets or feedback sources.

You'll run everything in a notebook or script, validate outputs, and understand the design decisions behind each component.

## Setup & Installation

Start by installing dependencies. Pin versions to ensure stability.

In [ ]:
!pip install -U crewai==0.28.0 crewai-tools==0.12.0 python-dotenv==1.0.0 pyyaml==6.0.1 pandas==2.2.0 matplotlib==3.8.2

Set your API keys. If you're in a notebook without a `.env` file, set them in-session:

In [ ]:
import os

os.environ["OPENAI_API_KEY"] = "your-openai-key-here"
os.environ["ANTHROPIC_API_KEY"] = "your-anthropic-key-here"  # optional, if using Claude

Verify the keys are present:

In [ ]:
required_keys = ["OPENAI_API_KEY"]
missing = [k for k in required_keys if not os.getenv(k)]
if missing:
    raise EnvironmentError(
        f"Missing required environment variables: {', '.join(missing)}\n"
        "Please set them before running the notebook."
    )
print("All required API keys found.")

Create the project directories:

In [ ]:
from pathlib import Path

ROOT = Path.cwd()
DATA_DIR = ROOT / "data"
CONFIGS_DIR = ROOT / "configs"
REPORTS_DIR = ROOT / "reports"
TOOLS_DIR = ROOT / "tools"

for d in [DATA_DIR, CONFIGS_DIR, REPORTS_DIR, TOOLS_DIR]:
    d.mkdir(parents=True, exist_ok=True)

Generate a sample CSV if you don't have one:

In [ ]:
csv_path = DATA_DIR / "customer_feedback.csv"
if not csv_path.exists():
    csv_path.write_text(
        "id,text\n"
        "1,\"I love the new dashboard—clean and fast.\"\n"
        "2,\"App crashed twice during checkout. Very frustrating.\"\n"
        "3,\"Support was helpful, but resolution took too long.\"\n"
        "4,\"Works as expected. No issues so far.\"\n",
        encoding="utf-8",
    )
    print(f"Created example CSV at {csv_path}")

## Step-by-Step Implementation

### Define Tools

Tools are functions agents call to perform actions. We'll create a file reader, a sentiment aggregator, and a chart generator.

Write the tools module to disk:

In [ ]:
tools_code = '''# tools/__init__.py
from crewai_tools import FileReadTool, tool
import json
from pathlib import Path
import matplotlib.pyplot as plt
import pandas as pd

def make_file_reader(csv_path: str):
    """Create a FileReadTool for reading the CSV."""
    return FileReadTool(file_path=csv_path, description="Read customer_feedback.csv")

@tool("aggregate_sentiment")
def aggregate_sentiment(jsonl_str: str) -> str:
    """
    Parse JSONL sentiment classifications and compute counts.
    
    Args:
        jsonl_str: JSONL string with 'sentiment' field per line.
    
    Returns:
        JSON string with keys 'positive', 'neutral', 'negative'.
    """
    lines = [ln.strip() for ln in jsonl_str.strip().split("\\n") if ln.strip()]
    records = [json.loads(ln) for ln in lines]
    df = pd.DataFrame(records)
    counts = df["sentiment"].value_counts().to_dict()
    result = {
        "positive": counts.get("positive", 0),
        "neutral": counts.get("neutral", 0),
        "negative": counts.get("negative", 0),
    }
    return json.dumps(result)

@tool("make_sentiment_pie")
def make_sentiment_pie(summary_json: str) -> str:
    """
    Generate a pie chart PNG from sentiment summary JSON.
    
    Args:
        summary_json: JSON with keys 'positive', 'neutral', 'negative'.
    
    Returns:
        Absolute path to the saved PNG.
    """
    data = json.loads(summary_json)
    counts = [data.get("positive", 0), data.get("neutral", 0), data.get("negative", 0)]
    labels = ["Positive", "Neutral", "Negative"]
    colors = ["#2ecc71", "#95a5a6", "#e74c3c"]
    
    reports_dir = Path("reports")
    reports_dir.mkdir(parents=True, exist_ok=True)
    out_path = reports_dir / "sentiment_pie.png"
    
    fig, ax = plt.subplots(figsize=(5, 5), dpi=160)
    ax.pie(counts, labels=labels, autopct="%1.1f%%", startangle=140, colors=colors)
    ax.axis("equal")
    plt.title("Sentiment Distribution")
    plt.tight_layout()
    fig.savefig(out_path)
    plt.close(fig)
    return str(out_path.resolve())
'''

tools_init = TOOLS_DIR / "__init__.py"
tools_init.write_text(tools_code, encoding="utf-8")
print(f"Tools module written to {tools_init}")

The `aggregate_sentiment` tool parses JSONL output from the classifier and computes counts using pandas, ensuring deterministic results. The `make_sentiment_pie` tool generates a PNG chart from the aggregated JSON.

### Configure Agents

Agents are defined in YAML. Each has a role, goal, backstory, and assigned tools.

In [ ]:
agents_yaml = """
sentiment_classifier:
  role: Sentiment Classifier
  goal: Label each feedback entry as positive, neutral, or negative
  backstory: You are an expert at understanding customer tone and intent. You read feedback text and assign accurate sentiment labels.
  llm: gpt-4o-mini
  tools:
    - file_reader
  verbose: true
  allow_delegation: false

aggregator:
  role: Data Aggregator
  goal: Compute sentiment counts from classified feedback
  backstory: You parse structured data and produce accurate numeric summaries.
  llm: gpt-4o-mini
  tools:
    - aggregate_sentiment
  verbose: true
  allow_delegation: false

chart_maker:
  role: Chart Generator
  goal: Create a pie chart visualizing sentiment distribution
  backstory: You turn data into clear, professional visualizations.
  llm: gpt-4o-mini
  tools:
    - make_sentiment_pie
  verbose: true
  allow_delegation: false

report_writer:
  role: Report Writer
  goal: Assemble a final Markdown report with insights and visuals
  backstory: You synthesize analysis into clear, actionable reports for stakeholders.
  llm: gpt-4o-mini
  tools: []
  verbose: true
  allow_delegation: false
"""

agents_path = CONFIGS_DIR / "agents.yaml"
agents_path.write_text(agents_yaml, encoding="utf-8")
print(f"Agents config written to {agents_path}")

Each agent is specialized: the classifier reads and labels, the aggregator computes counts, the chart maker visualizes, and the writer composes the final document.

### Configure Tasks

Tasks define what each agent does and how outputs flow between them.

In [ ]:
tasks_yaml = """
classify_feedback:
  description: |
    Read the customer_feedback.csv file using the file_reader tool.
    For each row, classify the sentiment of the 'text' field as positive, neutral, or negative.
    Output one JSON object per line (JSONL format) with fields: id, text, sentiment.
    Example:
    {"id": 1, "text": "I love it", "sentiment": "positive"}
    {"id": 2, "text": "It's okay", "sentiment": "neutral"}
  agent: sentiment_classifier
  expected_output: JSONL with id, text, and sentiment for each feedback entry.

aggregate_counts:
  description: |
    Take the JSONL output from the previous task.
    Use the aggregate_sentiment tool to compute the total counts of positive, neutral, and negative sentiments.
    Return a JSON object with keys: positive, neutral, negative.
  agent: aggregator
  expected_output: JSON object with sentiment counts.
  context:
    - classify_feedback

generate_chart:
  description: |
    Take the JSON summary from the previous task.
    Use the make_sentiment_pie tool to generate a pie chart PNG.
    Return the absolute file path to the saved chart.
  agent: chart_maker
  expected_output: Absolute path to sentiment_pie.png.
  context:
    - aggregate_counts

write_report:
  description: |
    Using the sentiment counts and chart path from previous tasks, write a Markdown report.
    Include:
    - A summary of total feedback entries and sentiment breakdown.
    - A table showing positive, neutral, and negative counts.
    - An embedded image of the pie chart using Markdown syntax: ![Sentiment Distribution](reports/sentiment_pie.png)
    - Brief insights or recommendations based on the sentiment distribution.
    Save nothing yourself; just return the Markdown content.
  agent: report_writer
  expected_output: Complete Markdown report with table, chart, and insights.
  context:
    - aggregate_counts
    - generate_chart
"""

tasks_path = CONFIGS_DIR / "tasks.yaml"
tasks_path.write_text(tasks_yaml, encoding="utf-8")
print(f"Tasks config written to {tasks_path}")

The `context` field wires dependencies: each task receives outputs from prior tasks, ensuring the pipeline runs in order. As your pipeline grows, be mindful of issues like [context rot](/article/context-rot-why-llms-forget-as-their-memory-grows-3), where LLMs may lose track of earlier information as more data accumulates.

### Instantiate Agents and Tasks

Load the YAML configs and build the agent and task objects.

In [ ]:
import yaml
from crewai import Agent, Task, LLM
from dotenv import load_dotenv

load_dotenv()

def load_yaml_config(path: Path) -> dict:
    """Load and parse a YAML file."""
    if not path.exists():
        raise FileNotFoundError(f"Missing config: {path}")
    with path.open("r", encoding="utf-8") as f:
        return yaml.safe_load(f) or {}

def get_llm(model_name: str | None) -> LLM:
    """Instantiate an LLM for the specified model."""
    model = model_name or os.getenv("DEFAULT_LLM_MODEL", "gpt-4o-mini")
    return LLM(model=model)

def build_tools_registry(csv_path: Path):
    """Build a registry of available tools."""
    from tools import make_file_reader, aggregate_sentiment, make_sentiment_pie
    return {
        "file_reader": make_file_reader(str(csv_path)),
        "aggregate_sentiment": aggregate_sentiment,
        "make_sentiment_pie": make_sentiment_pie,
    }

def instantiate_agents(agents_cfg: dict, tools_registry: dict) -> dict:
    """Instantiate Agent objects from YAML config."""
    agents = {}
    for key, spec in agents_cfg.items():
        role = spec.get("role")
        goal = spec.get("goal")
        backstory = spec.get("backstory", "")
        verbose = bool(spec.get("verbose", True))
        allow_delegation = bool(spec.get("allow_delegation", False))
        llm = get_llm(spec.get("llm"))
        tool_names = spec.get("tools", []) or []
        tools = [tools_registry[name] for name in tool_names if name in tools_registry]
        
        if not role or not goal:
            raise ValueError(f"Agent {key} missing role or goal")
        
        agents[key] = Agent(
            role=role,
            goal=goal,
            backstory=backstory,
            verbose=verbose,
            allow_delegation=allow_delegation,
            llm=llm,
            tools=tools,
        )
    return agents

def instantiate_tasks(tasks_cfg: dict, agents: dict) -> dict:
    """Instantiate Task objects from YAML config."""
    tasks = {}
    for key, spec in tasks_cfg.items():
        description = spec.get("description", "").strip()
        expected_output = spec.get("expected_output", "").strip()
        agent_key = spec.get("agent")
        if agent_key not in agents:
            raise KeyError(f"Task {key} references unknown agent '{agent_key}'")
        tasks[key] = Task(
            description=description,
            agent=agents[agent_key],
            expected_output=expected_output,
        )
    for key, spec in tasks_cfg.items():
        ctx = spec.get("context", []) or []
        context_tasks = [tasks[name] for name in ctx if name in tasks]
        if context_tasks:
            tasks[key].context = context_tasks
    return tasks

agents_cfg = load_yaml_config(agents_path)
tasks_cfg = load_yaml_config(tasks_path)

tools_registry = build_tools_registry(csv_path)
agents = instantiate_agents(agents_cfg, tools_registry)
tasks = instantiate_tasks(tasks_cfg, agents)

print(f"Instantiated {len(agents)} agents and {len(tasks)} tasks.")

This code loads the YAML, attaches tools to agents, and wires task dependencies. Each agent gets only the tools it needs, and tasks reference prior outputs via context.

### Run the Crew

Assemble the crew and execute the pipeline sequentially.

In [ ]:
from crewai import Crew, Process

crew = Crew(
    agents=list(agents.values()),
    tasks=list(tasks.values()),
    process=Process.sequential,
    verbose=True,
)

print("Kicking off crew...")
final_output = crew.kickoff()
print("\n=== Final Output (Report) ===\n")
print(final_output)

The crew runs each task in order, passing context forward. You'll see logs for each agent's actions and tool calls.

### Save and Validate Outputs

Write the final report to disk and verify the chart was created.

In [ ]:
report_path = REPORTS_DIR / "customer_feedback_report.md"
report_path.write_text(str(final_output), encoding="utf-8")
print(f"\nReport saved to: {report_path.resolve()}")

chart_path = REPORTS_DIR / "sentiment_pie.png"
if chart_path.exists():
    print(f"Chart saved to: {chart_path.resolve()}")
else:
    print("Warning: Chart file not found. Check tool execution logs.")

Open the Markdown file to see the full report, including the sentiment table and embedded chart. If running in a notebook, display the chart inline:

In [ ]:
from IPython.display import Image, display

if chart_path.exists():
    display(Image(filename=str(chart_path)))

## Key Design Decisions

**Why deterministic aggregation?** Asking an LLM to compute counts risks hallucination or inconsistency. A Python tool using pandas guarantees correct totals every time.

**Why sequential execution?** Dependencies are clear: you can't aggregate before classifying, or chart before aggregating. Sequential processing keeps the pipeline simple and debuggable. You can parallelize independent tasks later if needed.

**Why YAML for config?** Externalizing agent and task definitions makes the system maintainable. You can swap models, adjust prompts, or add agents without touching orchestration code.

**Why separate tools for each step?** Each tool has a single responsibility. This modularity lets you test, reuse, and replace components independently.

## Conclusion

You've built a complete Customer Feedback Analyst with CrewAI. You defined agents in YAML, wired tasks with context dependencies, attached tools for file reading, aggregation, and charting, and ran the system end-to-end to produce a real Markdown report and PNG visualization.

This architecture is reusable: swap the CSV for another dataset, adjust the classifier prompt, or add agents for deeper analysis. You now have a template for multi-agent pipelines that coordinate specialized roles to deliver structured, validated outputs.

**Next steps:**

- Add schema validation for JSONL outputs using Pydantic to catch malformed data early.
- Implement chunking for large CSVs to avoid context limits.
- Integrate observability: log token usage, task timings, and tool calls for cost and performance tracking.
- Explore parallel task execution for independent steps to reduce latency.
- Deploy the pipeline as an API or scheduled job for continuous feedback analysis.